In [1]:
import pandas as pd
from pathlib import Path

In [2]:
datapath1 = Path('/content/drive/MyDrive/json150')
datapath2 = Path('/content/drive/MyDrive/json_115')

In [3]:
file_list1= list(datapath2.glob('*.json'))
file_list2= list(datapath2.glob('*.json'))
file_list = file_list1 + file_list2
len(file_list),file_list[0]


(188,
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_13_49_18.json'))

In [4]:
import json
json_data=[]
good_file=[]
bad_file=[]
for file in file_list:
    try:
        # print(file)
        f = open(file)
        data = json.load(f)
        json_data.append(data)
        good_file.append(file)
    except:
        bad_file.append(file)
    f.close()

print(len(json_data))
# print(len(bad_file))
print(f"Good files: {len(good_file)}, Bad files: {len(bad_file)}")
# json_data

186
Good files: 186, Bad files: 2


In [5]:
list(bad_file)

[PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Sep_22_10_37_32_timestamped.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Sep_22_10_37_32_timestamped.json')]

In [6]:
##above bad_file do not contain any data

In [7]:
df=pd.json_normalize(json_data)

#combining all data to data frame

In [8]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,CallContent.Seq52.ReceiverContent,CallContent.Seq52.Timestamp,CallContent.Seq53.ReceiverContent,CallContent.Seq53.Timestamp,CallContent.Seq54.CallerContent,CallContent.Seq54.ReceiverContent,CallContent.Seq54.Timestamp,CallContent.Seq11.OtherSpeakerContent,CallContent.Seq5.OtherSpeakerContent,Content
0,value,Emily,Crafton Family Practice (Kumaran),2023_Dec_04_13:49:18,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,value,no_name,Crafton Family Practice (dominic),2023_Dec_04_14:46:53,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023_Dec_04_15:24:59,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,value,Ed Jones,Dominic (Crofton Family Practice),2023_12_04_14:59:14,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023_12_04_14:51:12,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df=pd.json_normalize(json_data)
df['Timestamp']=df['Timestamp'].str.replace("_","-",2)
df['Timestamp']=df['Timestamp'].str.replace("_"," ",1)
df['Timestamp']=df['Timestamp'].str.replace("_",":",2)
df['Timestamp']=pd.to_datetime(df['Timestamp'],format='mixed',errors='coerce')
df['Timestamp']=pd.to_datetime(df['Timestamp']).dt.normalize()
df['call_weekday'] = df['Timestamp'].dt.day_name()
df['call_month'] = df['Timestamp'].dt.month_name()
df['CallerContent']=df.loc[:,(df.columns.str.endswith('CallerContent'))].apply(lambda x: ' '.join(x.dropna().astype(str).values),axis=1)
df['ReceiverContent']=df.loc[:,(df.columns.str.endswith('ReceiverContent'))].apply(lambda x: ' '.join(x.dropna().astype(str).values),axis=1)

print(df.isna().sum())
print(df[['CallerContent','ReceiverContent']])
df.sample(10)


Metadata                           54
Caller                             50
Receiver                           10
Timestamp                          14
CallContent.Seq1.CallerContent     88
                                 ... 
Content                           178
call_weekday                       14
call_month                         14
CallerContent                       0
ReceiverContent                     0
Length: 185, dtype: int64
                                         CallerContent  \
0    Hi, I just want to double check. My daughter h...   
1    last known thanks for calling corruption from ...   
2    I'd like to speak out with the front office ab...   
3    Hi, this is Ed Jones. I'm a patient at Dr. Wil...   
4    Hello? Yes, I'd like to make an appointment fo...   
..                                                 ...   
181                                                      
182  You sent me to the front office. She put me on...   
183                               

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,CallContent.Seq54.CallerContent,CallContent.Seq54.ReceiverContent,CallContent.Seq54.Timestamp,CallContent.Seq11.OtherSpeakerContent,CallContent.Seq5.OtherSpeakerContent,Content,call_weekday,call_month,CallerContent,ReceiverContent
110,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
34,NaN,Randy,Dominic,2024-03-27,"Hi, I'm a new patient and I'd like to schedule...",Have you registered with Crawford Family Pract...,2024-03-27 00:04:00,"I do not think so. Well, I know I haven't.","Yes. For new patients, we will send an email l...",2024-03-27 00:18:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,March,"Hi, I'm a new patient and I'd like to schedule...",Have you registered with Crawford Family Pract...
137,value,Chris,Dominic,2023-09-20,NaN,Good afternoon. Thanks for calling Crofton Fam...,2023-09-20 16:03:09,"Hi, my name's Chris. I work with Dr. Sayan. I'...","Yeah, can I give you a minute, ma'am? Let me t...",2023-09-20 16:03:15,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,September,"Hi, my name's Chris. I work with Dr. Sayan. I'...",Good afternoon. Thanks for calling Crofton Fam...
135,value,Camelia Watson,Dominic,2023-09-20,"Hi, Dominic. I'm trying to reach the office an...","Okay. Kindly give me a minute, ma'am. Let me t...",2023-09-20 15:21:05,NaN,NaN,2023-09-20 15:21:18,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,September,"Hi, Dominic. I'm trying to reach the office an...","Okay. Kindly give me a minute, ma'am. Let me t..."
149,value,NaN,Dominic from Crofton Family Practice,2023-09-21,NaN,Good morning. Thank you for calling Crofton Fa...,2023-09-21 00:00:00,"Oh, hi. My name is Leslie Bucallo, and I had t...","Okay. Okay, regarding that, let me transfer th...",2023-09-21 11:28:56,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,September,"Oh, hi. My name is Leslie Bucallo, and I had t...",Good morning. Thank you for calling Crofton Fa...
70,value,Judith,Dominic,2023-11-01,"Hi, good afternoon. This is Judith again. Can ...","Yeah, sure. One moment. If the call is not ans...",2023-11-01 14:29:03,No problem.,Your conversation is being recorded. Thank you...,2023-11-01 14:29:17,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,November,"Hi, good afternoon. This is Judith again. Can ...","Yeah, sure. One moment. If the call is not ans..."
53,value,Anonymous,Dominic,2023-09-21,NaN,Good morning. Thanks for calling Crosswinds Fa...,2023-09-21 09:13:18,Who am I speaking? Good morning.,"Hi, this is Kumar.",2023-09-21 09:13:08,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,September,"Who am I speaking? Good morning. Kumar? Yes, c...",Good morning. Thanks for calling Crosswinds Fa...
35,NaN,Michael,Kumar,2024-03-27,Good morning. I'd like to schedule a physical ...,For you both. One moment. Could I take your da...,2024-03-27 00:10:00,I think last year's was in December.,You have made your previous physical. It was o...,2024-03-27 00:40:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,March,Good morning. I'd like to schedule a physical ...,For you both. One moment. Could I take your da...
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...
184,value,Allison,Kumar,2023-10-26,"Hi, yes. I'd like to make an appointment with ...",Good morning. Thanks for calling Carlson Famil...,2023-10-26 10:04:16,A televisit would be fine.,Would you like to meet the doctor in person or...,2023-10-26 10:04:25,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,October,"Hi, yes. I'd like to make an appointment with ...",Good mor

In [10]:
df_n1 = pd.DataFrame(columns=['ts'])
df_n1['ts'] = df['CallContent.Seq1.Timestamp']
df_n1



,ts
0,2023_Dec_04_13:49:18
1,2023_Dec_04_14:46:53
2,2023_Dec_04_15:24:59
3,2023_12_04_14:59:14
4,2023_12_04_14:51:12
...,...
181,NaN
182,2023-09-20 10:03:04
183,NaN
184,2023-10-26 10:04:16


In [11]:
df_n1['ts']=df_n1['ts'].str.replace("_","-",2)
df_n1['ts']=df_n1['ts'].str.replace("_"," ",1)
df_n1['ts']=df_n1['ts'].str.replace("_",":",2)
df_n1['ts']=pd.to_datetime(df_n1['ts'],format='mixed')

In [13]:
df_n1

,ts
0,2023-12-04 13:49:18
1,2023-12-04 14:46:53
2,2023-12-04 15:24:59
3,2023-12-04 14:59:14
4,2023-12-04 14:51:12
...,...
181,NaT
182,2023-09-20 10:03:04
183,NaT
184,2023-10-26 10:04:16


In [24]:
df_n1['Timezone'] = df_n1['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.hour
df_n1_grouped = df_n1.groupby('Timezone')['ts'].count()
df_n1_grouped_morning = df_n1_grouped[0:9].sum()
df_n1_grouped_afternoon = df_n1_grouped[9:17].sum()
df_n1_grouped_evening = df_n1_grouped[17:24].sum()

print("Total calls in morning:", df_n1_grouped_morning)
print("Total calls in  afternoon:", df_n1_grouped_afternoon)
print("Total calls in evening", df_n1_grouped_evening)


Total calls in morning: 56
Total calls in  afternoon: 64
Total calls in evening 56


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Columns: 185 entries, Metadata to ReceiverContent
dtypes: datetime64[ns](1), object(184)
memory usage: 269.0+ KB


In [26]:
df.shape

(186, 185)

In [27]:
df.columns

Index(['Metadata', 'Caller', 'Receiver', 'Timestamp',
       'CallContent.Seq1.CallerContent', 'CallContent.Seq1.ReceiverContent',
       'CallContent.Seq1.Timestamp', 'CallContent.Seq2.CallerContent',
       'CallContent.Seq2.ReceiverContent', 'CallContent.Seq2.Timestamp',
       ...
       'CallContent.Seq54.CallerContent', 'CallContent.Seq54.ReceiverContent',
       'CallContent.Seq54.Timestamp', 'CallContent.Seq11.OtherSpeakerContent',
       'CallContent.Seq5.OtherSpeakerContent', 'Content', 'call_weekday',
       'call_month', 'CallerContent', 'ReceiverContent'],
      dtype='object', length=185)

In [28]:
len(df.columns)

185

In [29]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,CallContent.Seq54.CallerContent,CallContent.Seq54.ReceiverContent,CallContent.Seq54.Timestamp,CallContent.Seq11.OtherSpeakerContent,CallContent.Seq5.OtherSpeakerContent,Content,call_weekday,call_month,CallerContent,ReceiverContent
0,value,Emily,Crafton Family Practice (Kumaran),2023-12-04,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic..."
1,value,no_name,Crafton Family Practice (dominic),2023-12-04,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023-12-04,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,"Hello? Yes, I'd like to make an appointment fo...","Good afternoon, this is Kumaran from Family Pr..."


#Number of calls by person  making the calls

In [30]:
print(df.Caller.value_counts(ascending = False))


Caller
Anonymous                   8
Judith                      6
Marilyn Hardy               6
No name                     6
Unnamed                     4
Cheryl Naiman               4
Robert                      4
No Name                     4
John Hopkins Imaging        2
Chris                       2
Michael                     2
Roberta Nelson              2
Alan Smith                  2
Sandra Arrington            2
Ms. Watson                  2
Dr. Williams                2
Camelia Watson              2
Mr. Leatherman              2
No Name (Beverly Cantor)    2
Latoya                      2
Allison                     2
Faith Van Lange             2
Alan                        2
Randy                       2
Ken Brown                   2
Unknown                     2
Ellen                       2
William Taylor              2
Madonna Simon               2
Emily                       2
no_name                     2
Hartwell Alderson           2
Brian Fryer                 2
Ed 

In [31]:
print(df.Receiver.value_counts())

Receiver
Dominic                                  62
Kumar                                    24
Dominic (Crofton Family Practice)        10
Kathy                                     8
Dominic from Crofton Family Practice      8
Kumaran (Crofton Family Practice)         6
Dominique                                 6
Brandi                                    4
Brandy                                    4
Kathy from Crofton Family Practice        4
Kumar from Crofton Family Practice        4
Kathy from Crosswind Family Practice      4
Kumar from Crawford Family Practice       2
Dominic from Robson Family Practice       2
No Name                                   2
Dominic from Grobson Family Practice      2
Dominic from Coffin Family Practice       2
Dominic from CrossFit Family Practice     2
Dominic from Grafton Family Practice      2
Crafton Family Practice (Kumaran)         2
Kathy from Proud and Family Practice      2
Crafton Family Practice (dominic)         2
Dominic (Crossroads Fam

#Sentiment analysis on caller content


In [33]:
from transformers import pipeline
sentiment_model = pipeline("sentiment-analysis", truncation=True, max_length=512, model='distilbert/distilbert-base-uncased-finetuned-sst-2-english')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [34]:
#Sentiment Analysis of Caller Data
df['sent_model']= df['CallerContent'].apply(sentiment_model)
df['Caller_Sentiment']= df['sent_model'].apply(lambda s: s[0]['label'])
df['Caller_Sentiment_Score']= df['sent_model'].apply(lambda s: s[0]['score'])
print(df[['CallerContent','Caller_Sentiment','Caller_Sentiment_Score']])

                                         CallerContent Caller_Sentiment  \
0    Hi, I just want to double check. My daughter h...         POSITIVE   
1    last known thanks for calling corruption from ...         NEGATIVE   
2    I'd like to speak out with the front office ab...         POSITIVE   
3    Hi, this is Ed Jones. I'm a patient at Dr. Wil...         NEGATIVE   
4    Hello? Yes, I'd like to make an appointment fo...         NEGATIVE   
..                                                 ...              ...   
181                                                            POSITIVE   
182  You sent me to the front office. She put me on...         NEGATIVE   
183                                                            POSITIVE   
184  Hi, yes. I'd like to make an appointment with ...         POSITIVE   
185  Good morning. My name is Mary Shealy, and I ne...         NEGATIVE   

     Caller_Sentiment_Score  
0                  0.998999  
1                  0.991963  
2        

In [35]:
df.Caller_Sentiment.value_counts()

Caller_Sentiment
POSITIVE    102
NEGATIVE     84
Name: count, dtype: int64

In [36]:
df.Caller_Sentiment_Score.describe()

count    186.000000
mean       0.919813
std        0.117845
min        0.513014
25%        0.861991
50%        0.991574
75%        0.996564
max        0.999823
Name: Caller_Sentiment_Score, dtype: float64

In [37]:
##*Emotional analysis of caller content*##
# model= "pysentimiento/robertuito-sentiment-analysis"
model="bhadresh-savani/distilbert-base-uncased-emotion"
emotion_model = pipeline(model=model, truncation=True, max_length=512)

In [38]:
data = ["I love you", "I hate you"]
emotion_model(data)

[{'label': 'love', 'score': 0.9608993530273438},
 {'label': 'anger', 'score': 0.8268487453460693}]

In [39]:
df['emot_model']= df['CallerContent'].apply(emotion_model)
df['Caller_Emotion']= df['emot_model'].apply(lambda s: s[0]['label'])
df['Caller_Emotion_Score']= df['emot_model'].apply(lambda s: s[0]['score'])
print(df[['CallerContent','Caller_Emotion','Caller_Emotion_Score']])

                                         CallerContent Caller_Emotion  \
0    Hi, I just want to double check. My daughter h...            joy   
1    last known thanks for calling corruption from ...            joy   
2    I'd like to speak out with the front office ab...            joy   
3    Hi, this is Ed Jones. I'm a patient at Dr. Wil...            joy   
4    Hello? Yes, I'd like to make an appointment fo...            joy   
..                                                 ...            ...   
181                                                             anger   
182  You sent me to the front office. She put me on...          anger   
183                                                             anger   
184  Hi, yes. I'd like to make an appointment with ...            joy   
185  Good morning. My name is Mary Shealy, and I ne...            joy   

     Caller_Emotion_Score  
0                0.998286  
1                0.985529  
2                0.995210  
3          

In [40]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,call_weekday,call_month,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score
0,value,Emily,Crafton Family Practice (Kumaran),2023-12-04,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,Monday,December,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...","[{'label': 'POSITIVE', 'score': 0.998999297618...",POSITIVE,0.998999,"[{'label': 'joy', 'score': 0.9982855916023254}]",joy,0.998286
1,value,no_name,Crafton Family Practice (dominic),2023-12-04,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,Monday,December,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,"[{'label': 'NEGATIVE', 'score': 0.991962909698...",NEGATIVE,0.991963,"[{'label': 'joy', 'score': 0.9855291247367859}]",joy,0.985529
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023-12-04,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,Monday,December,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,"[{'label': 'POSITIVE', 'score': 0.995154619216...",POSITIVE,0.995155,"[{'label': 'joy', 'score': 0.9952101111412048}]",joy,0.995210
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,Monday,December,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,"[{'label': 'NEGATIVE', 'score': 0.998019695281...",NEGATIVE,0.998020,"[{'label': 'joy', 'score': 0.9964357614517212}]",joy,0.996436
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,Monday,December,"Hello? Yes, I'd like to make an appointment fo...","Good afternoon, this is Kumaran from Family Pr...","[{'label': 'NEGATIVE', 'score': 0.994520127773...",NEGATIVE,0.994520,"[{'label': 'joy', 'score': 0.9134501218795776}]",joy,0.913450


In [41]:
df.describe()

,Timestamp,Caller_Sentiment_Score,Caller_Emotion_Score
count,172,186.000000,186.000000
mean,2023-11-07 11:43:15.348837120,0.919813,0.850522
min,2023-09-20 00:00:00,0.513014,0.327407
25%,2023-09-21 00:00:00,0.861991,0.768901
50%,2023-09-21 00:00:00,0.991574,0.962100
75%,2023-12-04 00:00:00,0.996564,0.995210
max,2024-10-29 00:00:00,0.999823,0.998739
std,NaN,0.117845,0.199422


In [42]:
#Calls per weekday
df.groupby(df.call_weekday).count()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,Content,call_month,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score
call_weekday,,,,,,,,,,,,,,,,,,,,,
Monday,22,26,26,26,26,26,26,26,26,26,...,0,26,26,26,26,26,26,26,26,26
Thursday,64,26,66,66,10,64,66,60,60,66,...,0,66,66,66,66,66,66,66,66,66
Tuesday,0,4,4,4,2,2,2,2,2,2,...,2,4,4,4,4,4,4,4,4,4
Wednesday,46,76,76,76,60,74,74,70,72,74,...,2,76,76,76,76,76,76,76,76,76


In [43]:
df['call_weekday'].unique()

array(['Monday', 'Wednesday', nan, 'Thursday', 'Tuesday'], dtype=object)

In [44]:
df.call_weekday.value_counts()

call_weekday
Wednesday    76
Thursday     66
Monday       26
Tuesday       4
Name: count, dtype: int64

In [45]:
df['call_month'].unique()

array(['December', 'November', nan, 'March', 'October', 'February',
       'September'], dtype=object)

In [46]:
df.groupby(df.call_month).count()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,Content,call_weekday,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score
call_month,,,,,,,,,,,,,,,,,,,,,
December,22,22,22,22,22,22,22,22,22,22,...,0,22,22,22,22,22,22,22,22,22
February,0,2,2,2,2,2,2,2,2,2,...,0,2,2,2,2,2,2,2,2,2
March,0,22,22,22,20,20,20,20,20,20,...,2,22,22,22,22,22,22,22,22,22
November,20,24,24,24,24,24,24,24,24,24,...,0,24,24,24,24,24,24,24,24,24
October,8,10,10,10,10,10,10,10,10,10,...,0,10,10,10,10,10,10,10,10,10
September,82,52,92,92,20,88,90,80,82,90,...,2,92,92,92,92,92,92,92,92,92


In [85]:
#day wise total number of calls
df_grouped = df.groupby('Timestamp')
df_grouped = df_grouped.apply(lambda x: x.groupby(pd.Grouper(key='Timestamp', freq='8H')).size())
df_grouped

Timestamp   Timestamp 
2023-09-20  2023-09-20    34
2023-09-21  2023-09-21    58
2023-10-26  2023-10-26     8
2023-11-01  2023-11-01    18
2023-11-15  2023-11-15     4
2023-11-20  2023-11-20     2
2023-12-04  2023-12-04    22
2024-02-12  2024-02-12     2
2024-03-19  2024-03-19     2
2024-03-27  2024-03-27    20
2024-10-29  2024-10-29     2
dtype: int64

In [84]:
# groupby timestamp in different months and count total number of calls

df_grouped_months = df.groupby(['call_month','Timestamp']).count()
df_grouped_months


Metadata  Caller  Receiver  \
call_month Timestamp                                
December   2023-12-04        22      22        22   
February   2024-02-12         0       2         2   
March      2024-03-19         0       2         2   
           2024-03-27         0      20        20   
November   2023-11-01        18      18        18   
           2023-11-15         2       4         4   
           2023-11-20         0       2         2   
October    2023-10-26         8       8         8   
           2024-10-29         0       2         2   
September  2023-09-20        26      34        34   
           2023-09-21        56      18        58   

                       CallContent.Seq1.CallerContent  \
call_month Timestamp                                    
December   2023-12-04                              22   
February   2024-02-12                               2   
March      2024-03-19                               0   
           2024-03-27                              20   
November   2023-11-01                              18   
           2023-11-15                               4   
           2023-11-20                               2   
October    2023-10-26                               8   
           2024-10-29                               2   
September  2023-09-20                              18   
           2023-09-21                               2   

                       CallContent.Seq1.ReceiverContent  \
call_month Timestamp                                      
December   2023-12-04                                22   
February   2024-02-12                                 2   
March      2024-03-19                                 0   
           2024-03-27                                20   
November   2023-11-01                                18   
           2023-11-15                                 4   
           2023-11-20                                 2   
October    2023-10-26                                 8   
           2024-10-29                                 2   
September  2023-09-20                                32   
           2023-09-21                                56   

                       CallContent.Seq1.Timestamp  \
call_month Timestamp                                
December   2023-12-04                          22   
February   2024-02-12                           2   
March      2024-03-19                           0   
           2024-03-27                          20   
November   2023-11-01                          18   
           2023-11-15                           4   
           2023-11-20                           2   
October    2023-10-26                           8   
           2024-10-29                           2   
September  2023-09-20                          32   
           2023-09-21                          58   

                       CallContent.Seq2.CallerContent  \
call_month Timestamp                                    
December   2023-12-04                              22   
February   2024-02-12                               2   
March      2024-03-19                               0   
           2024-03-27                              20   
November   2023-11-01                              18   
           2023-11-15                               4   
           2023-11-20                               2   
October    2023-10-26                               8   
           2024-10-29                               2   
September  2023-09-20                              28   
           2023-09-21                              52   

                       CallContent.Seq2.ReceiverContent  \
call_month Timestamp                                      
December   2023-12-04                                22   
February   2024-02-12                                 2   
March      2024-03-19                                 0   
           2024-03-27                                20   
November   2023-11-01                

In [49]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,call_weekday,call_month,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score
0,value,Emily,Crafton Family Practice (Kumaran),2023-12-04,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,Monday,December,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...","[{'label': 'POSITIVE', 'score': 0.998999297618...",POSITIVE,0.998999,"[{'label': 'joy', 'score': 0.9982855916023254}]",joy,0.998286
1,value,no_name,Crafton Family Practice (dominic),2023-12-04,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,Monday,December,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,"[{'label': 'NEGATIVE', 'score': 0.991962909698...",NEGATIVE,0.991963,"[{'label': 'joy', 'score': 0.9855291247367859}]",joy,0.985529
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023-12-04,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,Monday,December,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,"[{'label': 'POSITIVE', 'score': 0.995154619216...",POSITIVE,0.995155,"[{'label': 'joy', 'score': 0.9952101111412048}]",joy,0.995210
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,Monday,December,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,"[{'label': 'NEGATIVE', 'score': 0.998019695281...",NEGATIVE,0.998020,"[{'label': 'joy', 'score': 0.9964357614517212}]",joy,0.996436
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,Monday,December,"Hello? Yes, I'd like to make an appointment fo...","Good afternoon, this is Kumaran from Family Pr...","[{'label': 'NEGATIVE', 'score': 0.994520127773...",NEGATIVE,0.994520,"[{'label': 'joy', 'score': 0.9134501218795776}]",joy,0.913450


In [51]:
df['CallContent.Seq1.Timestamp'].value_counts()

CallContent.Seq1.Timestamp
2023-09-21 00:00:00    24
2024-03-27 00:04:00     8
2023-10-26 10:04:16     6
2024-03-27 00:05:00     6
2023-11-01 14:29:05     4
                       ..
2023-09-20 16:08:10     2
2023-09-20 15:18:51     2
2023-09-20 15:10:46     2
2023-09-20 16:08:46     2
2024-10-29 00:04:00     2
Name: count, Length: 62, dtype: int64

#Sentiment analysis on receiver call


In [52]:
df['sent_model']= df['ReceiverContent'].apply(sentiment_model)
df['Receiver_Sentiment']= df['sent_model'].apply(lambda s: s[0]['label'])
df['Receiver_Sentiment_Score']= df['sent_model'].apply(lambda s: s[0]['score'])
print(df[['ReceiverContent','Receiver_Sentiment','Receiver_Sentiment_Score']])

                                       ReceiverContent Receiver_Sentiment  \
0    Good afternoon, this is Crafton Family Practic...           POSITIVE   
1    is this you for the day hi I was wondering if ...           NEGATIVE   
2    Good afternoon. Thanks for calling Grafton Fam...           POSITIVE   
3    Good afternoon. Thanks for calling Crofton Fam...           POSITIVE   
4    Good afternoon, this is Kumaran from Family Pr...           POSITIVE   
..                                                 ...                ...   
181                                                              POSITIVE   
182  No, ma'am. It's not like that. Yes, she was on...           NEGATIVE   
183                                                              POSITIVE   
184  Good morning. Thanks for calling Carlson Famil...           POSITIVE   
185  Would you like to meet the doctor in person, o...           POSITIVE   

     Receiver_Sentiment_Score  
0                    0.997877  
1          

#Emotional analysis on receiver content

In [53]:
# model= "pysentimiento/robertuito-sentiment-analysis"
model="bhadresh-savani/distilbert-base-uncased-emotion"
emotion_model = pipeline(model=model, truncation=True, max_length=512)
emotion_model(data)

[{'label': 'love', 'score': 0.9608993530273438},
 {'label': 'anger', 'score': 0.8268487453460693}]

In [54]:
# Emotional Analysis of Caller Data
df['emot_model']= df['ReceiverContent'].apply(emotion_model)
df['Receiver_Emotion']= df['emot_model'].apply(lambda s: s[0]['label'])
df['Receiver_Emotion_Score']= df['emot_model'].apply(lambda s: s[0]['score'])
print(df[['ReceiverContent','Receiver_Emotion','Receiver_Emotion_Score']])

                                       ReceiverContent Receiver_Emotion  \
0    Good afternoon, this is Crafton Family Practic...              joy   
1    is this you for the day hi I was wondering if ...          sadness   
2    Good afternoon. Thanks for calling Grafton Fam...              joy   
3    Good afternoon. Thanks for calling Crofton Fam...              joy   
4    Good afternoon, this is Kumaran from Family Pr...              joy   
..                                                 ...              ...   
181                                                               anger   
182  No, ma'am. It's not like that. Yes, she was on...              joy   
183                                                               anger   
184  Good morning. Thanks for calling Carlson Famil...              joy   
185  Would you like to meet the doctor in person, o...              joy   

     Receiver_Emotion_Score  
0                  0.996388  
1                  0.989658  
2        

In [55]:
df.describe()

,Timestamp,Caller_Sentiment_Score,Caller_Emotion_Score,Receiver_Sentiment_Score,Receiver_Emotion_Score
count,172,186.000000,186.000000,186.000000,186.000000
mean,2023-11-07 11:43:15.348837120,0.919813,0.850522,0.934484,0.884345
min,2023-09-20 00:00:00,0.513014,0.327407,0.530524,0.356813
25%,2023-09-21 00:00:00,0.861991,0.768901,0.937856,0.863715
50%,2023-09-21 00:00:00,0.991574,0.962100,0.988659,0.987782
75%,2023-12-04 00:00:00,0.996564,0.995210,0.998161,0.996786
max,2024-10-29 00:00:00,0.999823,0.998739,0.999828,0.998645
std,NaN,0.117845,0.199422,0.108872,0.189503


In [56]:
df.Receiver_Emotion.value_counts()

Receiver_Emotion
joy        152
anger       18
sadness     12
fear         4
Name: count, dtype: int64

#length of call based on word count


In [57]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score,Receiver_Sentiment,Receiver_Sentiment_Score,Receiver_Emotion,Receiver_Emotion_Score
0,value,Emily,Crafton Family Practice (Kumaran),2023-12-04,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,"[{'label': 'POSITIVE', 'score': 0.997876644134...",POSITIVE,0.998999,"[{'label': 'joy', 'score': 0.9963880777359009}]",joy,0.998286,POSITIVE,0.997877,joy,0.996388
1,value,no_name,Crafton Family Practice (dominic),2023-12-04,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,"[{'label': 'NEGATIVE', 'score': 0.980291128158...",NEGATIVE,0.991963,"[{'label': 'sadness', 'score': 0.9896580576896...",joy,0.985529,NEGATIVE,0.980291,sadness,0.989658
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023-12-04,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,"[{'label': 'POSITIVE', 'score': 0.824704349040...",POSITIVE,0.995155,"[{'label': 'joy', 'score': 0.8701915144920349}]",joy,0.995210,POSITIVE,0.824704,joy,0.870192
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,"[{'label': 'POSITIVE', 'score': 0.996501684188...",NEGATIVE,0.998020,"[{'label': 'joy', 'score': 0.9637673497200012}]",joy,0.996436,POSITIVE,0.996502,joy,0.963767
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,"[{'label': 'POSITIVE', 'score': 0.999104082584...",NEGATIVE,0.994520,"[{'label': 'joy', 'score': 0.9980828762054443}]",joy,0.913450,POSITIVE,0.999104,joy,0.998083


In [58]:
print(df.Receiver.value_counts())

Receiver
Dominic                                  62
Kumar                                    24
Dominic (Crofton Family Practice)        10
Kathy                                     8
Dominic from Crofton Family Practice      8
Kumaran (Crofton Family Practice)         6
Dominique                                 6
Brandi                                    4
Brandy                                    4
Kathy from Crofton Family Practice        4
Kumar from Crofton Family Practice        4
Kathy from Crosswind Family Practice      4
Kumar from Crawford Family Practice       2
Dominic from Robson Family Practice       2
No Name                                   2
Dominic from Grobson Family Practice      2
Dominic from Coffin Family Practice       2
Dominic from CrossFit Family Practice     2
Dominic from Grafton Family Practice      2
Crafton Family Practice (Kumaran)         2
Kathy from Proud and Family Practice      2
Crafton Family Practice (dominic)         2
Dominic (Crossroads Fam

In [59]:
#length of call based on receiver content
df['ReceiverContent']

0      Good afternoon, this is Crafton Family Practic...
1      is this you for the day hi I was wondering if ...
2      Good afternoon. Thanks for calling Grafton Fam...
3      Good afternoon. Thanks for calling Crofton Fam...
4      Good afternoon, this is Kumaran from Family Pr...
                             ...                        
181                                                     
182    No, ma'am. It's not like that. Yes, she was on...
183                                                     
184    Good morning. Thanks for calling Carlson Famil...
185    Would you like to meet the doctor in person, o...
Name: ReceiverContent, Length: 186, dtype: object

In [60]:
df['totalwords_receiver'] = df['ReceiverContent'].str.count(' ')
df['totalwords_receiver']

0       82
1       58
2      294
3      143
4      135
      ... 
181      0
182     89
183      0
184    114
185    136
Name: totalwords_receiver, Length: 186, dtype: int64

In [61]:
df['totalwords_receiver'].describe()

count    186.000000
mean     160.860215
std      141.902791
min        0.000000
25%       76.000000
50%      134.000000
75%      195.000000
max      816.000000
Name: totalwords_receiver, dtype: float64

In [62]:
#length of call based on caller content
df['CallerContent']

0      Hi, I just want to double check. My daughter h...
1      last known thanks for calling corruption from ...
2      I'd like to speak out with the front office ab...
3      Hi, this is Ed Jones. I'm a patient at Dr. Wil...
4      Hello? Yes, I'd like to make an appointment fo...
                             ...                        
181                                                     
182    You sent me to the front office. She put me on...
183                                                     
184    Hi, yes. I'd like to make an appointment with ...
185    Good morning. My name is Mary Shealy, and I ne...
Name: CallerContent, Length: 186, dtype: object

In [63]:
df['totalwords_caller'] = df['CallerContent'].str.count(' ')
df['totalwords_caller'].describe()

count    186.000000
mean     111.043011
std       83.701647
min        0.000000
25%       50.000000
50%       91.000000
75%      161.000000
max      341.000000
Name: totalwords_caller, dtype: float64

In [64]:
##Sentiment by person making the call
df.Caller_Emotion.value_counts()

Caller_Emotion
joy         124
anger        36
sadness      20
fear          2
surprise      2
love          2
Name: count, dtype: int64

In [65]:
df_n =df[df['Caller_Emotion']=='anger']['Caller']

In [66]:
df_n.value_counts()

Caller
No name          6
Marilyn Hardy    6
Dale Carnett     2
Unnamed          2
Dr. Williams     2
Judith           2
Name: count, dtype: int64

In [67]:
df[df['Caller_Emotion']=='sadness']['Caller']

14             Robert
15       McKay Godwin
20      Cheryl Naiman
23             Brandy
33            No Name
34              Randy
55            Unknown
70             Judith
76             Latoya
79     William Taylor
107            Robert
108      McKay Godwin
113     Cheryl Naiman
116            Brandy
126           No Name
127             Randy
148           Unknown
163            Judith
169            Latoya
172    William Taylor
Name: Caller, dtype: object

In [68]:
print(df[df['Caller_Emotion']=='sadness']['Caller'].value_counts())


Caller
Robert            2
McKay Godwin      2
Cheryl Naiman     2
Brandy            2
No Name           2
Randy             2
Unknown           2
Judith            2
Latoya            2
William Taylor    2
Name: count, dtype: int64


In [69]:
df[df['Receiver_Emotion']=='fear']['Receiver'].value_counts()

Receiver
Dominique                               2
Dominic from Grafton Family Practice    2
Name: count, dtype: int64

In [70]:
df.columns.to_list()

['Metadata',
 'Caller',
 'Receiver',
 'Timestamp',
 'CallContent.Seq1.CallerContent',
 'CallContent.Seq1.ReceiverContent',
 'CallContent.Seq1.Timestamp',
 'CallContent.Seq2.CallerContent',
 'CallContent.Seq2.ReceiverContent',
 'CallContent.Seq2.Timestamp',
 'CallContent.Seq3.CallerContent',
 'CallContent.Seq3.ReceiverContent',
 'CallContent.Seq3.Timestamp',
 'CallContent.Seq4.CallerContent',
 'CallContent.Seq4.ReceiverContent',
 'CallContent.Seq4.Timestamp',
 'CallContent.Seq5.CallerContent',
 'CallContent.Seq5.ReceiverContent',
 'CallContent.Seq5.Timestamp',
 'CallContent.Seq6.CallerContent',
 'CallContent.Seq6.ReceiverContent',
 'CallContent.Seq6.Timestamp',
 'CallContent.Seq7.CallerContent',
 'CallContent.Seq7.ReceiverContent',
 'CallContent.Seq7.Timestamp',
 'CallContent.Seq8.CallerContent',
 'CallContent.Seq8.ReceiverContent',
 'CallContent.Seq8.Timestamp',
 'CallContent.Seq9.CallerContent',
 'CallContent.Seq9.ReceiverContent',
 'CallContent.Seq9.Timestamp',
 'CallContent.Seq10.Ca

##Counting most frequent words in caller and receiver content


In [72]:
from collections import Counter
receiver_content_words = []
for content in df['ReceiverContent']:
  receiver_content_words.extend(content.split())

receiver_content_word_counts = Counter(receiver_content_words)

most_common_words = receiver_content_word_counts.most_common(10)

print("Most common words in receiver content:")
for word, count in most_common_words:
  print(f"{word}: {count}")


Most common words in receiver content:
you: 1188
I: 810
the: 808
to: 766
a: 644
for: 582
and: 530
me: 446
is: 424
have: 390


In [76]:
import gensim
from gensim.parsing.preprocessing import preprocess_documents

# Remove stopwords from receiver_content using gensim
receiver_content_without_stopwords = []
for content in df['ReceiverContent']:
    processed_content = preprocess_documents([content])
    receiver_content_without_stopwords.append(processed_content[0])

# Determine the most common words
receiver_content_word_counts = Counter(word for content in receiver_content_without_stopwords for word in content)
most_common_words = receiver_content_word_counts.most_common(30)

print("Most common words in receiver content without stopwords:")
for word, count in most_common_words:
    print(f"{word}: {count}")


Most common words in receiver content without stopwords:
okai: 482
thank: 342
let: 312
ye: 242
dai: 232
practic: 204
bye: 198
famili: 196
call: 196
good: 190
yeah: 176
appoint: 162
transfer: 162
todai: 160
help: 150
great: 136
like: 128
know: 120
sir: 120
right: 118
assist: 102
sure: 102
moment: 100
onlin: 100
go: 100
kindli: 98
correct: 96
stai: 92
crofton: 90
leav: 90


In [87]:
# Remove stopwords from Caller using gensim
caller_content_without_stopwords = []
for content in df['CallerContent']:
    processed_content = preprocess_documents([content])
    caller_content_without_stopwords.append(processed_content[0])

# Determine the most common words
caller_content_word_counts = Counter(word for content in caller_content_without_stopwords for word in content)
most_common_words = caller_content_word_counts.most_common(30)

print("Most common words in receiver content without stopwords:")
for word, count in most_common_words:
    print(f"{word}: {count}")

Most common words in receiver content without stopwords:
okai: 292
thank: 246
ye: 206
yeah: 176
bye: 142
right: 134
need: 130
appoint: 126
like: 110
call: 108
william: 106
good: 96
want: 94
know: 84
patient: 68
morn: 68
work: 64
offic: 58
go: 58
todai: 58
sure: 56
brandi: 56
dai: 56
let: 54
email: 52
time: 50
talk: 48
prescript: 46
blood: 46
sir: 44


In [88]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score,Receiver_Sentiment,Receiver_Sentiment_Score,Receiver_Emotion,Receiver_Emotion_Score,totalwords_receiver,totalwords_caller
0,value,Emily,Crafton Family Practice (Kumaran),2023-12-04,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,0.998999,"[{'label': 'joy', 'score': 0.9963880777359009}]",joy,0.998286,POSITIVE,0.997877,joy,0.996388,82,65
1,value,no_name,Crafton Family Practice (dominic),2023-12-04,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,0.991963,"[{'label': 'sadness', 'score': 0.9896580576896...",joy,0.985529,NEGATIVE,0.980291,sadness,0.989658,58,44
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023-12-04,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,0.995155,"[{'label': 'joy', 'score': 0.8701915144920349}]",joy,0.995210,POSITIVE,0.824704,joy,0.870192,294,85
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,0.998020,"[{'label': 'joy', 'score': 0.9637673497200012}]",joy,0.996436,POSITIVE,0.996502,joy,0.963767,143,152
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,0.994520,"[{'label': 'joy', 'score': 0.9980828762054443}]",joy,0.913450,POSITIVE,0.999104,joy,0.998083,135,88


In [89]:
good_file

[PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_13_49_18.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_14_46_53.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_15_24_59.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_14_59_14.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_14_51_12.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_15_37_27.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_15_37_08.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_15_25_39.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Nov_01_08_50_15.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_16_07_23.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Dec_04_16_00_34.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Nov_01_09_23_00.json'),
 PosixPath('/content/drive/MyDrive/json_115/crhg_2023_Nov_01_09_

#Reading Records as rows for each sequence of calls

In [90]:
df.columns.str.strip()

Index(['Metadata', 'Caller', 'Receiver', 'Timestamp',
       'CallContent.Seq1.CallerContent', 'CallContent.Seq1.ReceiverContent',
       'CallContent.Seq1.Timestamp', 'CallContent.Seq2.CallerContent',
       'CallContent.Seq2.ReceiverContent', 'CallContent.Seq2.Timestamp',
       ...
       'Caller_Sentiment_Score', 'emot_model', 'Caller_Emotion',
       'Caller_Emotion_Score', 'Receiver_Sentiment',
       'Receiver_Sentiment_Score', 'Receiver_Emotion',
       'Receiver_Emotion_Score', 'totalwords_receiver', 'totalwords_caller'],
      dtype='object', length=197)

In [92]:
df1 = df
df1.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score,Receiver_Sentiment,Receiver_Sentiment_Score,Receiver_Emotion,Receiver_Emotion_Score,totalwords_receiver,totalwords_caller
0,value,Emily,Crafton Family Practice (Kumaran),2023-12-04,"Hi, I just want to double check. My daughter h...","Good afternoon, this is Crafton Family Practic...",2023_Dec_04_13:49:18,"Sure, January 1st, 1997.","Yeah, sure. Could I take the patient's date of...",2023_Dec_04_13:49:32,...,0.998999,"[{'label': 'joy', 'score': 0.9963880777359009}]",joy,0.998286,POSITIVE,0.997877,joy,0.996388,82,65
1,value,no_name,Crafton Family Practice (dominic),2023-12-04,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,0.991963,"[{'label': 'sadness', 'score': 0.9896580576896...",joy,0.985529,NEGATIVE,0.980291,sadness,0.989658,58,44
2,value,Brian Fryer,Kumaran (Grafton Family Practice),2023-12-04,I'd like to speak out with the front office ab...,Good afternoon. Thanks for calling Grafton Fam...,2023_Dec_04_15:24:59,"Okay, thank you.","Okay, one moment. Let me shout for you to the ...",2023_Dec_04_15:24:59,...,0.995155,"[{'label': 'joy', 'score': 0.8701915144920349}]",joy,0.995210,POSITIVE,0.824704,joy,0.870192,294,85
3,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,0.998020,"[{'label': 'joy', 'score': 0.9637673497200012}]",joy,0.996436,POSITIVE,0.996502,joy,0.963767,143,152
4,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,0.994520,"[{'label': 'joy', 'score': 0.9980828762054443}]",joy,0.913450,POSITIVE,0.999104,joy,0.998083,135,88


In [93]:
def files_df(file_list):
    df2 = []
    for file in file_list:
        df1=pd.read_json(file).reset_index()
        df1['data_file']=file.stem
        df1['CallContent.Seq1.Timestamp']=df1['CallContent.Seq1.Timestamp'].astype(str)
        df1['CallContent.Seq1.Timestamp']=df1['CallContent.Seq1.Timestamp'].str.replace("_","-",2)
        df1['CallContent.Seq1.Timestamp']=df1['CallContent.Seq1.Timestamp'].str.replace("_"," ",1)
        df1['CallContent.Seq1.Timestamp']=df1['CallContent.Seq1.Timestamp'].str.replace("_",":",2)
        df1['CallContent.Seq1.Timestamp']=pd.to_datetime(df1['CallContent.Seq1.Timestamp'],format='mixed')
        df1['CallContent.Seq1.Timestamp_1']=pd.to_datetime(df1['CallContent.Seq1.Timestamp'],format='mixed')
        df1['CallContent.Seq1.Timestamp']=pd.to_datetime(df1['CallContent.Seq1.Timestamp']).dt.normalize()
        # df.rename(columns={'Timestamp':'Time_Start'},inplace=True)
        # df['CallerContent'] = df.CallContent['CallerContent']
        df2.append(df1)
    df1= pd.concat(df2)
    return df1

df1 = files_df(good_file)
df1.head()

KeyError: 'CallContent.Seq1.Timestamp'

#Business Insights

In [ ]:
#Productivity of call center employee - based on number of calls by employee
##(good and bad files combined)
df.Receiver.value_counts(ascending= False)